# Utilities.py

In [30]:
import os
import numpy
import argparse
import sys
import soundfile
import numpy as np
import librosa
import h5py
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import logging

import config
    

# Crea un directorio si no existe.
def create_folder(fd):
    if not os.path.exists(fd):
        os.makedirs(fd)
   
# Devuelve el nombre del archivo sin la extensión a partir de una ruta.   
def get_filename(path):
    path = os.path.realpath(path)
    na_ext = path.split('/')[-1]
    na = os.path.splitext(na_ext)[0]
    return na
   
   
# Configura el registro (logging) del programa.
# Crea un directorio para logs y un archivo log numerado.
# Configura la salida del log tanto en archivo como en consola.
def create_logging(log_dir, filemode):
    
    create_folder(log_dir)
    i1 = 0
    
    while os.path.isfile(os.path.join(log_dir, "%04d.log" % i1)):
        i1 += 1
        
    log_path = os.path.join(log_dir, "%04d.log" % i1)
    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                        datefmt='%a, %d %b %Y %H:%M:%S',
                        filename=log_path,
                        filemode=filemode)
                
    # Print to console   
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)
    
    return logging
   
# Lee un archivo de audio y opcionalmente lo resamplea a una frecuencia objetivo.
def read_audio(path, target_fs=None):

    (audio, fs) = soundfile.read(path)

    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)

    if target_fs is not None and fs != target_fs:
        audio = librosa.resample(audio, orig_sr=fs, target_sr=target_fs)
        fs = target_fs

    return audio, fs


# Calcula la media y desviación estándar de un array.
def calculate_scalar(x):

    if x.ndim == 2:
        axis = 0
        
    elif x.ndim == 3:
        axis = (0, 1)

    mean = np.mean(x, axis=axis)
    std = np.std(x, axis=axis)

    return mean, std


# Normaliza un array usando la media y la desviación estándar proporcionadas.
def scale(x, mean, std):

    return (x - mean) / std

# Desnormaliza un array utilizando la media y desviación estándar proporcionadas.
def inverse_scale(x, mean, std):

    return x * std + mean

# Repite una secuencia de entrada hasta alcanzar un número específico de pasos de tiempo.
def repeat_seq(x, time_steps):
    repeat_num = time_steps // len(x) + 1
    repeat_x = np.tile(x, (repeat_num, 1))[0 : time_steps]
    return repeat_x
    
# Calcula el Accuracy de una clasificación comparando las salidas con los objetivos.
def calculate_accuracy(output, target):
    acc = np.sum(output == target) / float(len(target))
    return acc
    
# Imprime y retorna la precisión por clase.
def print_class_wise_accuracy(output, target):
    """Print class wise accuracy."""
    
    global labels
    global ix_to_lb
    
    correctness = np.zeros(len(labels), dtype=np.int32)
    total = np.zeros(len(labels), dtype=np.int32)
  
    for n in range(len(target)):
        
        total[target[n]] += 1
        
        if output[n] == target[n]:
            correctness[target[n]] += 1
        
    class_wise_accuracy = correctness.astype(np.float32) / total
    
    logging.info('{:<30}{}/{}\t{}'.format(
        'event labels', 'correct', 'total', 'accuracy'))
        
    for (n, label) in enumerate(labels):
        logging.info('{:<30}{}/{}\t\t{:.2f}'.format(
            label, correctness[n], total[n], class_wise_accuracy[n]))
        
    class_wise_accuracy = np.array(class_wise_accuracy)
    
    return class_wise_accuracy, correctness, total

# Grafica la precisión por clase.
def plot_class_wise_accuracy(class_wise_accuracy):
    """Plot accuracy."""
    
    labels = config.labels
    classes_num = len(labels)
    
    fig, ax = plt.subplots(1, 1, figsize=(7.5, 4))
    ax.bar(np.arange(classes_num), class_wise_accuracy, alpha=0.5)
    ax.set_ylabel('Accuracy')
    ax.set_xlim(0, classes_num)
    ax.set_ylim(0., 1.)
    ax.xaxis.set_ticks(np.arange(classes_num))
    ax.xaxis.set_ticklabels(labels, rotation=90)
    plt.tight_layout()
    plt.show()
    
# Escribe los resultados de una prueba en un archivo CSV para su envío.
def write_testing_data_submission_csv(submission_path, audio_names, 
                                      sorted_indices):
    
    global kmax
    global ix_to_lb
    global corrupted_files
    
    # Write result to submission csv
    f = open(submission_path, 'w')
    
    f.write('fname,label\n')
    
    for (n, audio_name) in enumerate(audio_names):
        
        f.write('{},'.format(audio_name))
        
        predicted_labels = [ix_to_lb[sorted_indices[n, k]] for k in range(kmax)]
        
        f.write(' '.join(predicted_labels))
            
        f.write('\n')
    
    for audio_name in corrupted_files:
        f.write('{},{}\n'.format(audio_name, 'Acoustic_guitar'))
    
    f.close()
    
    print("Write result to {}".format(submission_path))

# average_precision 

In [2]:
import numpy as np

# Average Precision at k (Precisión Promedio en k): Esta función calcula cuán precisos son los primeros k elementos predichos en una lista, 
# comparándolos con una lista de elementos reales que deberían ser predichos, sin importar el orden en el que aparecen en la lista real.
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

# Mean Average Precision at k (Precisión Promedio Media en k): Esta función calcula el promedio de la precisión promedio en k (apk) 
# para múltiples listas de elementos predichos y reales.
def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

# data_generator

In [28]:
import h5py
import numpy as np
import time
import pandas as pd
import logging
import matplotlib.pyplot as plt
import config


class DataGenerator(object):
    
    # Propósito: Inicializa el generador de datos.
    # Funcionalidad: Carga datos desde un archivo HDF5, divide los datos en conjuntos de entrenamiento y validación opcional basado en un archivo CSV, 
    # y calcula la media y la desviación estándar de los datos de entrenamiento para normalización.
    def __init__(self, hdf5_path, batch_size, time_steps, 
        validation_csv=None, holdout_fold=None, seed=1234):
        """
        Inputs:
          hdf5_path: str, path of hdf5 data
          batch_size: int
          time_stes: int, number of frames of a logmel spectrogram patch
          validate_csv: string | None, if None then use all data for training
          holdout_fold: int
          seed: int, random seed
        """
        
        # Parameters
        self.batch_size = batch_size
        self.random_state = np.random.RandomState(seed)
        self.validate_random_state = np.random.RandomState(0)

        global labels
        self.labels = labels
        global lb_to_ix
        
        self.time_steps = time_steps
        self.hop_frames = self.time_steps // 2
        
        self.classes_num = len(self.labels)
        
        # Load data
        load_time = time.time()
        hf = h5py.File(hdf5_path, 'r')
            
        self.audio_names = np.array([s.decode() for s in hf['filename'][:]])
        self.x = hf['feature'][:]
        self.bgn_fin_indices = hf['bgn_fin_indices'][:]
        event_labels = hf['label'][:]
        self.y = np.array([lb_to_ix[s.decode()] for s in event_labels])
        self.manually_verifications = hf['manually_verification'][:]
        
        hf.close()
        
        logging.info('Loading data time: {:.3f} s'.format(
            time.time() - load_time))
        
        # Load validation
        if validation_csv:
            self.train_audio_indexes, self.validate_audio_indexes = \
                self.get_audio_indexes(validation_csv, holdout_fold)
                
        else:
            self.train_audio_indexes = np.arange(len(self.audio_names))
            self.validate_audio_indexes = np.array([])
                               
        logging.info('Training audios number: {}'.format(
            len(self.train_audio_indexes)))
            
        logging.info('Validation audios number: {}'.format(
            len(self.validate_audio_indexes)))
                    
        # calculate scalar
        (self.mean, self.std) = self.calculate_training_data_scalar()
    
        # Get training patches
        self.train_patch_bgn_fin_y_tuples = \
            self.calculate_patch_bgn_fin_y_tuples(self.train_audio_indexes)
        
        logging.info('Training patches number: {}'.format(
            len(self.train_patch_bgn_fin_y_tuples)))
    
    # Propósito: Obtiene índices de audios para entrenamiento y validación.
    # Funcionalidad: Lee un archivo CSV que especifica divisiones de datos (folds) y devuelve índices de audios para entrenamiento 
    # y validación según el fold especificado.
    def get_audio_indexes(self, validation_csv, holdout_fold):
        """Get train and audio indexes from validation csv. 
        """
        
        df = pd.read_csv(validation_csv, sep=',')
        df = pd.DataFrame(df)
        
        folds = df['fold']
        
        train_audio_indexes = np.where(folds != holdout_fold)[0]
        validate_audio_indexes = np.where(folds == holdout_fold)[0]
        
        return train_audio_indexes, validate_audio_indexes

    # Propósito: Calcula la media y la desviación estándar de los datos de entrenamiento.
    # Funcionalidad: Concatena todos los datos de entrenamiento y calcula la media y la desviación estándar, que se usarán 
    # para normalizar los datos durante el entrenamiento.
    def calculate_training_data_scalar(self):
        """Concatenate all training data and calculate scalar. 
        """
        
        train_bgn_fin_indices = self.bgn_fin_indices[self.train_audio_indexes]
        
        train_x_concat = []
        
        for [bgn, fin] in train_bgn_fin_indices:
            train_x_concat.append(self.x[bgn : fin])
            
        train_x_concat = np.concatenate(train_x_concat, axis=0)
        
        (mean, std) = calculate_scalar(train_x_concat)
        
        return mean, std
    
    # Propósito: Calcula las posiciones de inicio y fin de los parches de datos de entrenamiento.
    # Funcionalidad: Para cada audio en los índices dados, calcula los segmentos de datos que se usarán como parches de entrenamiento 
    # junto con sus etiquetas correspondientes.
    def calculate_patch_bgn_fin_y_tuples(self, audio_indexes):
        """Calculate (bgn, fin, y) tuples for selecting patches for training. 
        """
        
        bgn_fin_indices = self.bgn_fin_indices[audio_indexes]
        
        patch_bgn_fin_y_tuples = []

        for n in range(len(audio_indexes)):
            
            [bgn, fin] = bgn_fin_indices[n]
            y = self.y[audio_indexes[n]]

            patch_tuples_for_this_audio = \
                self.get_patch_bgn_fin_y_tuples_for_an_audio(bgn, fin, y)
                    
            patch_bgn_fin_y_tuples += patch_tuples_for_this_audio
            
        # Print class wise number of patches
        patches_per_class = np.zeros(self.classes_num, dtype=np.int32)
        
        for k in range(self.classes_num):
            patches_per_class[k] = np.sum(
                [tuple[2] == k for tuple in patch_bgn_fin_y_tuples])
        
        if False:
            for k in range(self.classes_num):
                logging.info('{:<30}{}'.format(
                    self.labels[k], patches_per_class[k]))
        
        return patch_bgn_fin_y_tuples
    
    # Propósito: Genera tuplas que especifican las posiciones de inicio y fin de los parches de datos dentro de un audio, junto con su etiqueta correspondiente.
    # Funcionalidad: Si la duración del audio es menor o igual a time_steps (número de fotogramas de un parche de espectrograma logmel), 
    # crea una única tupla que cubre todo el audio.
    # Si la duración del audio es mayor a time_steps, genera múltiples tuplas de parches de datos desplazados (hop_frames) a lo largo del audio.


    def get_patch_bgn_fin_y_tuples_for_an_audio(self, bgn, fin, y):
        """Get (bgn, fin, y) tuples in an audio. 
        """
        
        if fin - bgn <= self.time_steps:
            patch_tuples_for_this_audio = [(bgn, fin, y)]
            
        else:
            bgns = np.arange(bgn, fin - self.time_steps, self.hop_frames)
            patch_tuples_for_this_audio = []
            
            for bgn in bgns:
                patch_tuples_for_this_audio.append(
                    (bgn, bgn + self.time_steps, y))
                
        return patch_tuples_for_this_audio
    
    # Propósito: Genera lotes de datos de entrenamiento.
    # Funcionalidad: Itera sobre los parches de datos de entrenamiento, normaliza cada lote y los devuelve junto con sus etiquetas 
    # en lotes de tamaño especificado.
    def generate_train(self):
        
        batch_size = self.batch_size
        patch_bgn_fin_y_tuples = self.train_patch_bgn_fin_y_tuples.copy()
        time_steps = self.time_steps

        patches_num = len(patch_bgn_fin_y_tuples)

        self.random_state.shuffle(patch_bgn_fin_y_tuples)

        iteration = 0
        pointer = 0

        while True:

            # Reset pointer
            if pointer >= patches_num:
                pointer = 0
                self.random_state.shuffle(patch_bgn_fin_y_tuples)

            # Get batch indexes
            batch_patch_bgn_fin_y_tuples = patch_bgn_fin_y_tuples[
                pointer: pointer + batch_size]
                
            pointer += batch_size

            iteration += 1
            
            (batch_x, batch_y) = self.get_batch_x_y(
                self.x, batch_patch_bgn_fin_y_tuples)
            
            # Transform data
            batch_x = self.transform(batch_x)

            yield batch_x, batch_y
        
    # Propósito: Prepara lotes de datos y etiquetas para entrenamiento.
    # Funcionamiento: Esta función toma segmentos de datos full_x según las tuplas batch_patch_bgn_fin_y_tuples. 
    # Si un segmento es corto, lo repite hasta alcanzar la longitud deseada self.time_steps. Retorna los lotes de datos batch_x y las etiquetas batch_y.
    def get_batch_x_y(self, full_x, batch_patch_bgn_fin_y_tuples):
        """Get batch_x and batch_y, repeat is audio is short. 
        """
        
        batch_x = []
        batch_y = []
        
        for (bgn, fin, y) in batch_patch_bgn_fin_y_tuples:
            
            batch_y.append(y)
            
            if fin - bgn == self.time_steps:
                batch_x.append(full_x[bgn : fin])
                
            else:
                batch_x.append(repeat_seq(full_x[bgn : fin], self.time_steps))

        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        
        return batch_x, batch_y
    
    # Propósito: Genera lotes de datos para la validación del modelo.
    # Funcionamiento: Genera lotes de datos y etiquetas para validar el modelo durante el entrenamiento. 
    # Filtra y mezcla los datos según parámetros como data_type, manually_verified_only, y shuffle. Limita el número de 
    # audios procesados con max_audios_num. Retorna lotes de datos normalizados junto con las etiquetas y nombres de los audios asociados.
    def generate_validate_slices(self, data_type, manually_verified_only, 
                                 shuffle, max_audios_num=None):
        """Generate patches in an audio. 
        
        Args:
          data_type: 'train' | 'validate'
          manually_verified_only: bool
          shuffle: bool
          max_audios_num: int, set maximum audios to speed up validation
        """
        
        if data_type == 'train':
            audio_indexes = np.array(self.train_audio_indexes)
            
        elif data_type == 'validate':
            audio_indexes = np.array(self.validate_audio_indexes)
            
        else:
            raise Exception('Incorrect data_type!')
        
        if manually_verified_only:
            
            manually_verified_indexes = np.where(
                self.manually_verifications[audio_indexes]==1)[0]
                
            audio_indexes = audio_indexes[manually_verified_indexes]
        
        if shuffle:
            self.validate_random_state.shuffle(audio_indexes)
        
        for (n, audio_index) in enumerate(audio_indexes):
            
            if n == max_audios_num:
                break
            
            [bgn, fin] = self.bgn_fin_indices[audio_index]
            y = self.y[audio_index]
            audio_name = self.audio_names[audio_index]
            
            patch_tuples_for_this_audio = \
                self.get_patch_bgn_fin_y_tuples_for_an_audio(bgn, fin, y)
        
            (batch_x_for_an_audio, _) = self.get_batch_x_y(
                self.x, patch_tuples_for_this_audio)
        
            batch_x_for_an_audio = self.transform(batch_x_for_an_audio)

            yield batch_x_for_an_audio, y, audio_name
                
    #Propósito: Normaliza los datos de entrada.
    # Funcionamiento: Aplica una transformación de escala a los datos x utilizando la media y la desviación estándar calculadas previamente 
    # (self.mean y self.std). Retorna los datos normalizados.
    def transform(self, x):
        """Transform data. 
        
        Args:
          x: (batch_x, seq_len, freq_bins) | (seq_len, freq_bins)
          
        Returns:
          Transformed data. 
        """

        return scale(x, self.mean, self.std)
            
            
class TestDataGenerator(DataGenerator):
    
    # Carga los datos de prueba desde archivos HDF5 especificados por dev_hdf5_path y test_hdf5_path. 
    # También establece parámetros como time_steps para definir la longitud de los parches de datos y test_hop_frames para el paso de salto en la 
    # generación de lotes de prueba.
    def __init__(self, dev_hdf5_path, test_hdf5_path, time_steps, 
                 test_hop_frames):
        """Test data generator. 
        
        Args:
          dev_hdf5_path: str, path of development hdf5 file
          test_hdf5_path: str, path of test hdf5 file
          time_stes: int, number of frames of a logmel spectrogram patch
          test_hop_frames: int
        """
        
        super(TestDataGenerator, self).__init__(
            hdf5_path=dev_hdf5_path, 
            batch_size=None, 
            time_steps=time_steps,
            validation_csv=None)
        
        self.test_hop_frames = test_hop_frames
        
        global corrupted_files
        self.corrupted_files = corrupted_files
        
        # Load test data
        load_time = time.time()
        hf = h5py.File(test_hdf5_path, 'r')

        self.test_audio_names = np.array([s.decode() for s in hf['filename'][:]])
        self.test_x = hf['feature'][:]
        self.test_bgn_fin_indices = hf['bgn_fin_indices'][:]
        
        hf.close()
        
        logging.info('Loading data time: {:.3f} s'.format(
            time.time() - load_time))
    
    # Itera sobre los índices de los audios de prueba y genera lotes de datos normalizados junto con los nombres de los audios. 
    # Utiliza las funciones heredadas get_patch_bgn_fin_y_tuples_for_an_audio para obtener las tuplas de inicio y fin de los parches y 
    # get_batch_x_y para generar los lotes de datos. Los lotes de datos se normalizan usando transform. Cada iteración produce un lote de datos 
    # y el nombre del audio asociado.
    def generate_test_slices(self):
        
        test_hop_frames = self.test_hop_frames
        global corrupted_files
        corrupted_files = corrupted_files
        
        audio_indexes = range(len(self.test_audio_names))
        
        for (n, audio_index) in enumerate(audio_indexes):
            
            [bgn, fin] = self.test_bgn_fin_indices[audio_index]
            
            audio_name = self.test_audio_names[audio_index]
            
            if fin > bgn:
            
                patch_tuples_for_this_audio = \
                    self.get_patch_bgn_fin_y_tuples_for_an_audio(bgn, fin, y=None)
            
                (batch_x_for_an_audio, _) = \
                    self.get_batch_x_y(self.test_x, patch_tuples_for_this_audio)
            
                batch_x_for_an_audio = self.transform(batch_x_for_an_audio)
    
                yield batch_x_for_an_audio, audio_name

# Config

In [4]:
# Especifica la tasa de muestreo objetivo durante la extracción de características de audio. En este caso, es 32000 muestras por segundo.
sample_rate = 32000

# Son parámetros relacionados con la transformada de Fourier de corto tiempo (STFT). window_size define el tamaño de la ventana de la FFT 
# (2048 muestras en este caso), y overlap especifica la cantidad de solapamiento entre tramas sucesivas (1024 muestras).
window_size = 2048
overlap = 1024

# Es una variable que indica el número de bandas Mel que se utilizarán para la extracción de características de audio. 
# Las bandas Mel son intervalos de frecuencia no lineales que simulan cómo percibimos los tonos en términos de su "altura" tonal, siguiendo la escala Mel. 
# En este caso, se están utilizando 64 bandas Mel, lo que significa que durante la extracción de características de audio, se calcularán características 
# específicas en 64 intervalos de frecuencia diferentes dentro del espectro de audio.
mel_bins = 64

kmax = 3

# lista que contiene nombres de diferentes clases de sonidos. Cada elemento de esta lista representa una etiqueta de 
# clase específica asociada a diferentes tipos de sonidos.
labels = ['Acoustic_guitar', 'Applause', 'Bark', 'Bass_drum', 
          'Burping_or_eructation', 'Bus', 'Cello', 'Chime', 'Clarinet', 
          'Computer_keyboard', 'Cough', 'Cowbell', 'Double_bass', 
          'Drawer_open_or_close', 'Electric_piano', 'Fart', 'Finger_snapping', 
          'Fireworks', 'Flute', 'Glockenspiel', 'Gong', 'Gunshot_or_gunfire', 
          'Harmonica', 'Hi-hat', 'Keys_jangling', 'Knock', 'Laughter', 'Meow', 
          'Microwave_oven', 'Oboe', 'Saxophone', 'Scissors', 'Shatter', 
          'Snare_drum', 'Squeak', 'Tambourine', 'Tearing', 'Telephone', 
          'Trumpet', 'Violin_or_fiddle', 'Writing']

# ste diccionario mapea cada etiqueta (lb, que representa una categoría de sonido de la lista labels) a su índice 
# correspondiente (i). Utiliza la función enumerate para iterar sobre la lista labels, asignando un índice numérico a cada etiqueta.
lb_to_ix = {lb: i for i, lb in enumerate(labels)}

# Este diccionario hace lo contrario. Mapea cada índice (i) de la lista labels de vuelta a su etiqueta correspondiente (lb). 
# Es útil para convertir índices numéricos de clase en sus etiquetas de texto correspondientes.
ix_to_lb = {i: lb for i, lb in enumerate(labels)}

# Es una lista que contiene los nombres de archivo de audio que se consideran corruptos o problemáticos.
corrupted_files = ['0b0427e2.wav', '6ea0099f.wav', 'b39975f5.wav']

# create_validation

In [5]:
import os
import sys
import numpy as np
import pandas as pd
import argparse
import random

# crea una estructura de validación cruzada dividiendo los datos de entrenamiento en varios grupos llamados "pliegues". 
# Luego guarda esta información en un archivo CSV llamado validate_meta.csv.
def create_validation_folds(dataset_dir, workspace):
    """Create validation file with folds and write out to validate_meta.csv
    """
    random_state = np.random.RandomState(1234)
    folds_num = 4
    
    # Paths
    csv_path = os.path.join(dataset_dir, 'train.csv')
    
    # Read csv
    df = pd.DataFrame(pd.read_csv(csv_path))
    
    indexes = np.arange(len(df))
    random_state.shuffle(indexes)
    
    audios_num = len(df)
    audios_num_per_fold = int(audios_num // folds_num)

    # Create folds
    folds = np.zeros(audios_num, dtype=np.int32)
    
    for n in range(audios_num):
        folds[indexes[n]] = (n % folds_num) + 1

    df_ex = df
    df_ex['fold'] = folds

    # Write out validation csv
    out_path = os.path.join(workspace, 'validate_meta.csv')
    df_ex.to_csv(out_path)

    print("Write out to {}".format(out_path))


dataset_dir = "./Data"
workspace = "./"

create_validation_folds(dataset_dir, workspace)

Write out to ./validate_meta.csv


# features.py

In [6]:
import os
import sys
import numpy
import argparse
import sys
import soundfile
import numpy as np
import librosa
import h5py
import time
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import logging


class LogMelExtractor():

    # Propósito: Inicializa el extractor de características de LogMel.
    # Funcionalidad: Configura parámetros como el tamaño de la ventana de FFT, el solapamiento y las bandas Mel para 
    # la transformación de espectrogramas de audio a representaciones de LogMel.
    def __init__(self, sample_rate, window_size, overlap, mel_bins):
        
        self.window_size = window_size
        self.overlap = overlap
        self.ham_win = np.hamming(window_size)
        
        self.melW = librosa.filters.mel(sr=sample_rate, 
                                        n_fft=window_size, 
                                        n_mels=mel_bins, 
                                        fmin=50., 
                                        fmax=sample_rate // 2).T
    
    # Propósito: Transforma un fragmento de audio en características LogMel.
    # Funcionalidad: Calcula el espectrograma de magnitud y aplica la transformación LogMel utilizando filtros Mel específicos.
    def transform(self, audio):
    
        ham_win = self.ham_win
        window_size = self.window_size
        overlap = self.overlap
    
        [f, t, x] = signal.spectral.spectrogram(
                        audio, 
                        window=ham_win,
                        nperseg=window_size, 
                        noverlap=overlap, 
                        detrend=False, 
                        return_onesided=True, 
                        mode='magnitude') 
        x = x.T
            
        x = np.dot(x, self.melW)
        x = np.log(x + 1e-8)
        x = x.astype(np.float32)
        
        return x

# Propósito: Calcula características LogMel para un archivo de audio dado.
# Funcionalidad: Lee el archivo de audio, normaliza sus amplitudes, y utiliza el extractor LogMel para obtener las características LogMel del audio.
def calculate_logmel(audio_path, sample_rate, extractor):

    (audio, _) = read_audio(audio_path, target_fs=sample_rate)

    audio = audio / np.max(np.abs(audio))

    feature = extractor.transform(audio)

    return feature

# Propósito: Escribe características y metadatos de audios en un archivo HDF5.
# Funcionalidad: Lee metadatos de un archivo CSV, extrae características LogMel para cada audio, y escribe estas características 
# junto con otros metadatos en un archivo HDF5. Permite el uso de un subconjunto de datos (mini_data) para pruebas rápidas.
def calculate_features(dataset_dir, workspace, data_type, mini_data):
    """Write features and infos of audios to a hdf5 file.
    """

    global sample_rate
    global window_size
    global overlap
    global mel_bins

    global corrupted_files

    # Paths
    if data_type == 'development':
        audio_dir = os.path.join(dataset_dir, 'audio_train')
        meta_csv = os.path.join(dataset_dir, 'train.csv')
        
    elif data_type == 'test':
        audio_dir = os.path.join(dataset_dir, 'audio_test')
        meta_csv = os.path.join(dataset_dir, 'sample_submission.csv')

    if mini_data:
        hdf5_path = os.path.join(workspace, 'features', 'logmel', 
                                 'mini_{}.h5'.format(data_type))
    else:
        hdf5_path = os.path.join(workspace, 'features', 'logmel', 
                                 '{}.h5'.format(data_type))
    
    create_folder(os.path.dirname(hdf5_path))

    # Load csv
    df = pd.DataFrame(pd.read_csv(meta_csv))

    audio_names = []
    
    if data_type == 'development':
        manually_verifications = []
        target_labels = []

    for row in df.iterrows():
        
        audio_name = row[1]['fname']
        audio_names.append(audio_name)
        
        if data_type == 'development':
            
            target_label = row[1]['label']
            manually_verification = row[1]['manually_verified']
        
            target_labels.append(target_label)
            manually_verifications.append(manually_verification)

    # Use partial data when set mini_data to True
    if mini_data:
        
        audios_num = 300
        random_state = np.random.RandomState(0)
        audio_indexes = np.arange(len(audio_names))
        random_state.shuffle(audio_indexes)
        audio_indexes = audio_indexes[0 : audios_num]
        
        audio_names = [audio_names[idx] for idx in audio_indexes]
        
        if data_type == 'development':
            
            target_labels = [target_labels[idx] for idx in audio_indexes]
            
            manually_verifications = [manually_verifications[idx] 
                                      for idx in audio_indexes]
        
        print("Number of audios: {}".format(len(audio_names)))

    # Feature extractor
    extractor = LogMelExtractor(sample_rate=sample_rate,
                                window_size=window_size,
                                overlap=overlap,
                                mel_bins=mel_bins)

    # Write out to h5 file
    hf = h5py.File(hdf5_path, 'w')

    hf.create_dataset(
        name='feature',
        shape=(0, mel_bins),
        maxshape=(None, mel_bins),
        dtype=np.float32)

    calculate_time = time.time()
    bgn_fin_indices = []

    # Extract feature for audios
    for (n, audio_name) in enumerate(audio_names):

        print(n, audio_name)

        # Extract feature
        if audio_name in corrupted_files:
            feature = np.zeros((0, mel_bins))
            
        else:
            audio_path = os.path.join(audio_dir, audio_name)
            feature = calculate_logmel(audio_path, sample_rate, extractor)
        
        print(feature.shape)

        # Write feature to hdf5
        bgn_indice = hf['feature'].shape[0]
        fin_indice = bgn_indice + feature.shape[0]

        hf['feature'].resize((fin_indice, mel_bins))
        hf['feature'][bgn_indice: fin_indice] = feature

        bgn_fin_indices.append((bgn_indice, fin_indice))

    # Write infos to hdf5
    hf.create_dataset(name='filename', 
                      data=[s.encode() for s in audio_names], 
                      dtype='S32')
    
    hf.create_dataset(name='bgn_fin_indices',
                      data=bgn_fin_indices,
                      dtype=np.int32)
    
    if data_type == 'development':
        
        hf.create_dataset(name='label', 
                          data=[s.encode() for s in target_labels], 
                          dtype='S32')
        
        hf.create_dataset(name='manually_verification',
                          data=manually_verifications,
                          dtype=np.int32)
        
    hf.close()
    
    print("Write out hdf5 file to {}".format(hdf5_path))
    print("Time spent: {} s".format(time.time() - calculate_time))


data_type = 'development'
mini_data = False
calculate_features(dataset_dir, workspace, data_type, mini_data)


0 00044347.wav


C:\Users\gluar\AppData\Local\Temp\ipykernel_31964\1624066509.py:36: DeprecationWarning: Please import `spectrogram` from the `scipy.signal` namespace; the `scipy.signal.spectral` namespace is deprecated and will be removed in SciPy 2.0.0.
  [f, t, x] = signal.spectral.spectrogram(


(436, 64)
1 001ca53d.wav
(321, 64)
2 002d256b.wav
(12, 64)
3 0033e230.wav
(249, 64)
4 00353774.wav
(140, 64)
5 003b91e8.wav
(414, 64)
6 003da8e5.wav
(41, 64)
7 0048fd00.wav
(31, 64)
8 004ad66f.wav
(217, 64)
9 0063ab88.wav
(925, 64)
10 006f2f32.wav
(51, 64)
11 0075d39c.wav
(644, 64)
12 00780200.wav
(34, 64)
13 0079d310.wav
(125, 64)
14 0091fc7f.wav
(177, 64)
15 0097160c.wav
(81, 64)
16 00ad7068.wav
(134, 64)
17 00c5808a.wav
(484, 64)
18 00c82919.wav
(47, 64)
19 00c934d7.wav
(77, 64)
20 00c9e799.wav
(64, 64)
21 00cb787c.wav
(220, 64)
22 00ce569f.wav
(80, 64)
23 00d1fe46.wav
(139, 64)
24 00d3bba3.wav
(303, 64)
25 00d40fa2.wav
(98, 64)
26 00d9fa61.wav
(155, 64)
27 00e2b4cd.wav
(439, 64)
28 00f88dc5.wav
(249, 64)
29 00fbb28b.wav
(545, 64)
30 00fcbab2.wav
(12, 64)
31 010aa387.wav
(22, 64)
32 011a2185.wav
(194, 64)
33 0120d246.wav
(314, 64)
34 01235a12.wav
(194, 64)
35 01257aad.wav
(276, 64)
36 01302128.wav
(454, 64)
37 013264d3.wav
(554, 64)
38 013c3135.wav
(170, 64)
39 01506d76.wav
(565, 64

In [7]:
data_type = 'test'
calculate_features(dataset_dir, workspace, data_type, mini_data)

0 00063640.wav
(48, 64)
1 0013a1db.wav
(24, 64)
2 002bb878.wav
(40, 64)
3 002d392d.wav
(14, 64)
4 00326aa9.wav
(64, 64)
5 0038a046.wav
(120, 64)
6 003995fa.wav
(155, 64)
7 005ae625.wav
(177, 64)
8 007759c4.wav


C:\Users\gluar\AppData\Local\Temp\ipykernel_31964\1624066509.py:36: DeprecationWarning: Please import `spectrogram` from the `scipy.signal` namespace; the `scipy.signal.spectral` namespace is deprecated and will be removed in SciPy 2.0.0.
  [f, t, x] = signal.spectral.spectrogram(


(312, 64)
9 008afd93.wav
(294, 64)
10 00a161c0.wav
(207, 64)
11 00a7a2f6.wav
(135, 64)
12 00ae03f6.wav
(277, 64)
13 00b2404e.wav
(115, 64)
14 00beb030.wav
(412, 64)
15 00c4d5b8.wav
(245, 64)
16 00c92c05.wav
(129, 64)
17 00ccf065.wav
(9, 64)
18 00d0ab77.wav
(299, 64)
19 00dffe3a.wav
(9, 64)
20 00e33205.wav
(210, 64)
21 00e7ed07.wav
(241, 64)
22 00eac343.wav
(34, 64)
23 00f2f692.wav
(129, 64)
24 0102a895.wav
(183, 64)
25 010a0b3a.wav
(27, 64)
26 010eed01.wav
(14, 64)
27 01115622.wav
(24, 64)
28 01198b7e.wav
(358, 64)
29 01207ce5.wav
(20, 64)
30 0137bfba.wav
(249, 64)
31 014eb52e.wav
(154, 64)
32 01608fa3.wav
(20, 64)
33 0169348e.wav
(225, 64)
34 016e52da.wav
(217, 64)
35 0183b482.wav
(129, 64)
36 0185de6b.wav
(249, 64)
37 01a5a2a3.wav
(576, 64)
38 01aed32a.wav
(129, 64)
39 01afccc5.wav
(608, 64)
40 01b4cdb5.wav
(499, 64)
41 01b85217.wav
(235, 64)
42 01bb344f.wav
(61, 64)
43 01bede12.wav
(542, 64)
44 01c9d6d3.wav
(149, 64)
45 01e628cb.wav
(651, 64)
46 01e6e112.wav
(500, 64)
47 01e99e2d.wa

# model_pytorch

In [8]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F


# Propósito: Mover datos a la GPU si está disponible.
# Funcionamiento: Convierte los datos a tensores de PyTorch según su tipo (flotante o entero). Si se usa CUDA, mueve los tensores a la GPU.
def move_data_to_gpu(x, cuda):

    if 'float' in str(x.dtype):
        x = torch.Tensor(x)

    elif 'int' in str(x.dtype):
        x = torch.LongTensor(x)

    else:
        raise Exception("Error!")

    if cuda:
        x = x.cuda()

    return x

# Propósito: Inicializar capas lineales o de convolución.
# Funcionamiento: Calcula los parámetros de inicialización según la configuración de la capa y ajusta los pesos y sesgos de la capa de manera uniforme.
def init_layer(layer):
    """Initialize a Linear or Convolutional layer. 
    Ref: He, Kaiming, et al. "Delving deep into rectifiers: Surpassing 
    human-level performance on imagenet classification." Proceedings of the 
    IEEE international conference on computer vision. 2015.
    """
    
    if layer.weight.ndimension() == 4:
        (n_out, n_in, height, width) = layer.weight.size()
        n = n_in * height * width
        
    elif layer.weight.ndimension() == 2:
        (n_out, n) = layer.weight.size()

    std = math.sqrt(2. / n)
    scale = std * math.sqrt(3.)
    layer.weight.data.uniform_(-scale, scale)

    if layer.bias is not None:
        layer.bias.data.fill_(0.)


# Propósito: Inicializar capas de normalización por lotes.
# Funcionamiento: Establece los sesgos a 0 y los pesos a 1 para la capa de normalización por lotes.
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)
    

class BaselineCnn(nn.Module):
    # Propósito: Inicializar la arquitectura de la CNN básica.
    # Funcionamiento: Define cuatro capas de convolución y una capa totalmente conectada, junto con capas de normalización por lotes.
    def __init__(self, classes_num):
        
        super(BaselineCnn, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64,
                               kernel_size=(5, 5), stride=(2, 2),
                               padding=(2, 2), bias=False)

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128,
                               kernel_size=(5, 5), stride=(2, 2),
                               padding=(2, 2), bias=False)

        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256,
                               kernel_size=(5, 5), stride=(2, 2),
                               padding=(2, 2), bias=False)

        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512,
                               kernel_size=(5, 5), stride=(2, 2),
                               padding=(2, 2), bias=False)

        self.fc1 = nn.Linear(512, classes_num, bias=True)

        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)
        self.bn4 = nn.BatchNorm2d(512)

        self.init_weights()

    # Propósito: Inicializar los pesos y las capas de normalización por lotes.
    # Funcionamiento: Llama a las funciones init_layer y init_bn para cada capa.
    def init_weights(self):

        init_layer(self.conv1)
        init_layer(self.conv2)
        init_layer(self.conv3)
        init_layer(self.conv4)
        init_layer(self.fc1)

        init_bn(self.bn1)
        init_bn(self.bn2)
        init_bn(self.bn3)
        init_bn(self.bn4)

    # Propósito: Definir la propagación hacia adelante de la red.
    # Funcionamiento: Aplica las capas de convolución, activación ReLU, y max pooling. Luego, realiza una proyección log-softmax en la salida de la capa totalmente conectada.
    def forward(self, input, return_bottleneck=False):
        (_, seq_len, mel_bins) = input.shape

        x = input.view(-1, 1, seq_len, mel_bins)
        """(samples_num, feature_maps, time_steps, freq_num)"""
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        
        x = F.max_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0:2])

        x = F.log_softmax(self.fc1(x), dim=-1)

        return x
    

class VggishConvBlock(nn.Module):
    # Propósito: Definir la propagación hacia adelante de la red.
    # Funcionamiento: Aplica las capas de convolución, activación ReLU, y max pooling. Luego, realiza una proyección log-softmax en la salida de la capa totalmente conectada.
    def __init__(self, in_channels, out_channels):
        
        super(VggishConvBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.conv2 = nn.Conv2d(in_channels=out_channels, 
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)
                              
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.init_weights()
        
    # Propósito: Inicializar los pesos de las capas del bloque.
    # Funcionamiento: Llama a las funciones init_layer y init_bn para cada capa.
    def init_weights(self):
        
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)
    
    # Propósito: Definir la propagación hacia adelante del bloque.
    # Funcionamiento: Aplica dos capas de convolución, activación ReLU, y max pooling.
    def forward(self, input):
        
        x = input
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=(2, 2), stride=(2, 2))
        
        return x
    
    
class Vggish(nn.Module):
    # Propósito: Inicializar la arquitectura del modelo Vggish.
    # Funcionamiento: Define cuatro bloques de convolución estilo VGGish y una capa totalmente conectada.
    def __init__(self, classes_num):
        
        super(Vggish, self).__init__()

        self.conv_block1 = VggishConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = VggishConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = VggishConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = VggishConvBlock(in_channels=256, out_channels=512)

        self.fc_final = nn.Linear(512, classes_num, bias=True)

        self.init_weights()

    # Propósito: Inicializar los pesos de la capa totalmente conectada.
    # Funcionamiento: Llama a la función init_layer para la capa.
    def init_weights(self):

        init_layer(self.fc_final)

    # Propósito: Definir la propagación hacia adelante de la red Vggish.
    # Funcionamiento: Aplica los bloques de convolución y max pooling, luego realiza una proyección log-softmax en la salida de la capa totalmente conectada.
    def forward(self, input, return_bottleneck=False):
        (_, seq_len, mel_bins) = input.shape

        x = input.view(-1, 1, seq_len, mel_bins)
        '''(samples_num, feature_maps, time_steps, freq_num)'''

        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)

        x = F.max_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0:2])

        x = F.log_softmax(self.fc_final(x), dim=-1)

        return x

# main_pytorch

In [24]:
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))
import numpy as np
import time
import argparse
from sklearn import preprocessing
import matplotlib.pyplot as plt
import math
import h5py
import sys
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim



# Hyper-parameters
# Clase del modelo a utilizar, en este caso Vggish.
Model = Vggish
# Tamaño de los lotes de datos utilizados en el entrenamiento y evaluación.
batch_size = 128
# Número de pasos de tiempo en las entradas de los datos.
time_steps = 128
# Saltos de marcos en el entrenamiento.
train_hop_frames = 64
# Saltos de marcos en la prueba.
test_hop_frames = 16
    

# Propósito: Agregar las predicciones de fragmentos de clips de audio.
# Funcionamiento: Calcula la media de las predicciones a lo largo de los fragmentos y las retorna como un array.
def aggregate_outputs(outputs):
    """Aggregate the prediction of patches of audio clips. 
    
    Args:
      outputs: (audios_num, patches_num, classes_num)
    
    Returns:
      agg_outputs: (audios_num)
    """
    
    agg_outputs = []
    
    for output in outputs:
        agg_output = np.mean(output, axis=0)
        agg_outputs.append(agg_output)
    
    agg_outputs = np.array(agg_outputs)
    
    return agg_outputs
    
 
# Propósito: Evaluar el modelo en datos de entrenamiento o validación.
# Funcionamiento: Genera fragmentos de datos, realiza la inferencia del modelo, agrega las salidas, calcula las predicciones y mide la precisión y el mAP.
def evaluate(model, generator, data_type, cuda):
    """Evaluate
    
    Args:
      model: object.
      generator: object.
      data_type: 'train' | 'validate'.
      cuda: bool.
      
    Returns:
      accuracy: float
      mapk: float
    """

    if data_type == 'train':
        max_audios_num = 1000   # A small portion of training data to evaluate
    
    elif data_type == 'validate':
        max_audios_num = None   # All evaluation data to evaluate

    generate_func = generator.generate_validate_slices(
        data_type=data_type, 
        manually_verified_only=True, 
        shuffle=True, 
        max_audios_num=max_audios_num)
    
    # Forward
    dict = forward(model=model, 
                   generate_func=generate_func, 
                   cuda=cuda, 
                   return_target=True)

    outputs = dict['output']    # (audios_num, patches_num, classes_num)
    targets = dict['target']    # (audios_num,)

    agg_outputs = aggregate_outputs(outputs)
    '''(audios_num, classes_num)'''

    predictions = np.argmax(agg_outputs, axis=-1)
    '''(audios_num,)'''
    
    sorted_indices = np.argsort(agg_outputs, axis=-1)[:, ::-1][:, :kmax]
    '''(audios_num, kmax)'''

    # Accuracy
    accuracy = calculate_accuracy(predictions, targets)

    # mAP
    mapk_value = mapk(actual=[[e] for e in targets], 
                      predicted=[e.tolist() for e in sorted_indices], 
                      k=kmax)

    return accuracy, mapk_value
    
# Propósito: Pasar datos a través del modelo para obtener las predicciones.
# Funcionamiento: Procesa lotes de datos, realiza inferencia utilizando el modelo y recolecta las salidas y nombres de audio, y opcionalmente los objetivos.  
def forward(model, generate_func, cuda, return_target):
    """Forward data to a model.
    
    Args:
      generate_func: generate function
      cuda: bool
      return_target: bool
      
    Returns:
      dict, keys: 'audio_name', 'output'; optional keys: 'target'
    """
    
    outputs = []
    audio_names = []
    
    if return_target:
        targets = []
    
    # Evaluate on mini-batch
    for data in generate_func:
            
        if return_target:
            (batch_x_for_an_audio, y, audio_name) = data
            
        else:
            (batch_x_for_an_audio, audio_name) = data
            
        batch_x_for_an_audio = move_data_to_gpu(batch_x_for_an_audio, cuda)

        # Predict
        model.eval()
        outputs_for_an_audio = model(batch_x_for_an_audio)

        # Append data
        outputs.append(outputs_for_an_audio.data.cpu().numpy())
        audio_names.append(audio_name)
        
        if return_target:
            targets.append(y)

    dict = {}

    # outputs = np.array(outputs)
    outputs = np.asarray(outputs, dtype="object")
    dict['output'] = outputs
    
    audio_names = np.array(audio_names)
    dict['audio_name'] = audio_names
    
    if return_target:
        targets = np.array(targets)
        dict['target'] = targets
        
    return dict
    
# Propósito: Entrenar el modelo en los datos de entrenamiento.
# Funcionamiento: Configura los caminos de datos, inicializa el modelo y el optimizador, y entrena el modelo en lotes de datos, 
# evaluando periódicamente y guardando el estado del modelo.
def train(workspace, filename, validate, holdout_fold, cuda, mini_data):
    
    global labels
    num_classes = len(labels)

    # Use partial data for training
    if mini_data:
        hdf5_path = os.path.join(workspace, 'features', 'logmel',
                                 'mini_development.h5')        
        
    else:
        hdf5_path = os.path.join(workspace, 'features', 'logmel',
                                 'development.h5')
    
    if validate:
        validation_csv = os.path.join(workspace, 'validate_meta.csv')
        
        models_dir = os.path.join(workspace, 'models', filename, 
            'holdout_fold{}'.format(holdout_fold))
        
    else:
        validation_csv = None
        
        models_dir = os.path.join(workspace, 'models', filename, 'full_train')
    
    create_folder(models_dir)
    
    # Model
    model = Model(num_classes)
    
    if cuda:
        model.cuda()
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), 
                           eps=1e-08, weight_decay=0.)
    
    # Data generator
    generator = DataGenerator(hdf5_path=hdf5_path, 
                              batch_size=batch_size, 
                              time_steps=time_steps, 
                              validation_csv=validation_csv, 
                              holdout_fold=holdout_fold)

    iteration = 0
    train_bgn_time = time.time()

    # Train on mini batches
    for (batch_x, batch_y) in generator.generate_train():
        
        # Evaluate
        if iteration % 200 == 0:

            train_fin_time = time.time()
            
            (tr_acc, tr_mapk) = evaluate(model=model, 
                                         generator=generator, 
                                         data_type='train', 
                                         cuda=cuda)
                                         
            logging.info('train acc: {:.3f}, train mapk: {:.3f}'.format(
                tr_acc, tr_mapk))
                        
            if validate:
                (va_acc, va_mapk) = evaluate(model=model, 
                                             generator=generator, 
                                             data_type='validate', 
                                             cuda=cuda)
                                             
                logging.info('valid acc: {:.3f}, validate mapk: {:.3f}'.format(
                    va_acc, va_mapk))
        
            train_time = train_fin_time - train_bgn_time
            validate_time = time.time() - train_fin_time
            
            logging.info('------------------------------------')
            logging.info('Iteration: {}, train time: {:.3f} s, eval time: '
                '{:.3f} s'.format(iteration, train_time, validate_time))
            
            train_bgn_time = time.time()
            
        # Save model
        if iteration % 1000 == 0 and iteration > 0:
            save_out_dict = {'iteration': iteration, 
                             'state_dict': model.state_dict(), 
                             'optimizer': optimizer.state_dict(), }
            
            save_out_path = os.path.join(models_dir, 
                'md_{}_iters.tar'.format(iteration))
                
            torch.save(save_out_dict, save_out_path)
            logging.info('Save model to {}'.format(save_out_path))
            
        # Reduce learning rate
        if iteration % 100 == 0 and iteration > 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= 0.9
        
        batch_x = move_data_to_gpu(batch_x, cuda)
        batch_y = move_data_to_gpu(batch_y, cuda)
        
        # Forward
        t_forward = time.time()
        model.train()
        output = model(batch_x)
        
        # Loss
        loss = F.nll_loss(output, batch_y)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        iteration += 1
        
        # Stop learning
        if iteration == 10001:
            break
    
# Propósito: Realizar inferencia en los datos de validación.
# Funcionamiento: Carga el modelo guardado, genera fragmentos de datos de validación, 
# realiza la inferencia y calcula métricas de rendimiento, guardando los resultados.
def inference_validation_data(workspace, holdout_fold, iteration, filename, cuda):
    
    global labels
    classes_num = len(labels)
    
    # Paths    
    model_path = os.path.join(workspace, 'models', filename, 
                              'holdout_fold{}'.format(holdout_fold), 
                              'md_{}_iters.tar'.format(iteration))
    
    hdf5_path = os.path.join(workspace, 'features', 'logmel', 
                             'development.h5')
    
    validation_csv = os.path.join(workspace, 'validate_meta.csv')
    
    stats_pickle_path = os.path.join(workspace, 'stats', filename, 
                                     'holdout_fold{}'.format(holdout_fold), 
                                     '{}_iters.p'.format(iteration))
    
    create_folder(os.path.dirname(stats_pickle_path))
    
    # Model
    model = Model(classes_num)
        
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['state_dict'])
    
    if cuda:
        model.cuda()
    
    # Data generator
    generator = DataGenerator(hdf5_path=hdf5_path, 
                              batch_size=batch_size, 
                              time_steps=time_steps, 
                              validation_csv=validation_csv, 
                              holdout_fold=holdout_fold)
    
    generate_func = generator.generate_validate_slices(
        data_type='validate', 
        manually_verified_only=True, 
        shuffle=False, 
        max_audios_num=None)
    
    # Forward
    dict = forward(model=model, 
                   generate_func=generate_func, 
                   cuda=cuda, 
                   return_target=True)

    outputs = dict['output']    # (audios_num, patches_num, classes_num)
    targets = dict['target']    # (audios_num,)

    agg_outputs = aggregate_outputs(outputs)
    '''(audios_num, classes_num)'''

    predictions = np.argmax(agg_outputs, axis=-1)
    '''(audios_num,)'''
    
    sorted_indices = np.argsort(agg_outputs, axis=-1)[:, ::-1][:, :kmax]
    '''(audios_num, kmax)'''

    # Accuracy
    accuracy = calculate_accuracy(predictions, targets)

    # mAP
    mapk_value = mapk(actual=[[e] for e in targets], 
                      predicted=[e.tolist() for e in sorted_indices], 
                      k=kmax)
    
    # Print
    logging.info('')
    logging.info('iteration: {}'.format(iteration))
    logging.info('accuracy: {:.3f}'.format(accuracy))
    logging.info('mapk: {:.3f}'.format(mapk_value))
    
    (class_wise_accuracy, correctness, total) = print_class_wise_accuracy(
        predictions, targets)
        
    # Save stats for current holdout training
    dict = {'correctness': correctness, 'total': total, 
            'accuracy': accuracy, 'mapk': mapk_value}
    
    pickle.dump(dict, open(stats_pickle_path, 'wb'))
    
    logging.info('Write out stat to {}'.format(stats_pickle_path))
    

# Propósito: Realizar inferencia en los datos de prueba.
# Funcionamiento: Carga el modelo guardado, genera fragmentos de datos de prueba, realiza la inferencia y guarda las predicciones en un archivo CSV para la sumisión.
def inference_testing_data(workspace, iteration, filename, cuda):
    global labels

    num_classes = len(labels)
    
    # Paths
    model_path = os.path.join(workspace, 'models', filename, 'holdout_fold1', 
                              'md_{}_iters.tar'.format(iteration))
    
    dev_hdf5_path = os.path.join(workspace, 'features', 'logmel', 
                                 'development.h5')
                                 
    test_hdf5_path = os.path.join(workspace, 'features', 'logmel', 'test.h5')
    
    submission_path = os.path.join(workspace, 'submissions', filename, 
                                   'iteration={}'.format(iteration), 
                                   'submission.csv')
    
    create_folder(os.path.dirname(submission_path))
    
    # Model
    model = Model(num_classes)
        
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['state_dict'])
    
    if cuda:
        model.cuda()
    
    # Data generator
    test_generator = TestDataGenerator(dev_hdf5_path=dev_hdf5_path, 
                                       test_hdf5_path=test_hdf5_path, 
                                       test_hop_frames=test_hop_frames, 
                                       time_steps=time_steps)
    
    generate_func = test_generator.generate_test_slices()
    
    # Forward
    dict = forward(model=model, 
                   generate_func=generate_func, 
                   cuda=cuda, 
                   return_target=False)
    
    outputs = dict['output']
    audio_names = dict['audio_name']
    
    agg_outputs = aggregate_outputs(outputs)
    '''(audios_num, classes_num)'''

    predictions = np.argmax(agg_outputs, axis=-1)
    '''(audios_num,)'''
    
    sorted_indices = np.argsort(agg_outputs, axis=-1)[:, ::-1][:, :kmax]
    '''(audios_num, kmax)'''
    
    # Write out submission csv
    write_testing_data_submission_csv(submission_path, audio_names, 
                                      sorted_indices)

In [12]:
workspace = "./"
cuda = 0
holdout_fold = 1
filename = ""
validate = 1
mini_data = 0

train(workspace, filename, validate, holdout_fold, cuda, mini_data)

1000
1
1
1
1
1
1
2
1
2
1
1
8
2
4
1
7
1
2
4
8
1
1
3
1
2
1
1
1
1
7
2
1
1
1
5
8
1
1
3
1
1
1
1
1
1
1
1
1
2
10
1
1
1
1
2
1
1
4
1
1
1
8
1
1
2
1
3
1
1
1
1
2
1
1
8
1
1
1
1
2
1
1
1
4
1
2
1
1
1
7
1
3
1
1
1
1
1
1
1
2
1
1
1
1
1
1
2
1
6
2
1
3
1
5
1
1
1
1
1
1
1
2
1
2
1
2
1
3
3
1
1
1
1
1
1
1
1
1
1
7
1
1
1
1
2
1
1
9
1
1
1
1
1
1
1
8
1
1
1
1
1
1
1
1
1
1
13
2
1
1
4
4
1
1
5
6
1
1
1
9
1
1
1
1
3
4
4
1
1
1
1
1
1
1
1
1
10
1
1
1
1
1
1
1
1
2
1
1
11
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
3
3
3
1
1
1
1
1
5
1
1
4
1
1
2
1
2
1
1
1
1
1
2
1
1
3
1
1
1
11
1
1
1
1
1
3
6
1
1
1
1
1
1
10
1
1
3
1
1
1
1
2
5
1
6
1
1
9
4
1
1
1
6
1
1
11
1
2
1
1
2
1
1
7
1
1
1
1
1
1
1
1
1
2
1
3
1
2
1
2
10
1
8
1
1
1
3
3
9
1
11
7
1
1
2
1
2
1
4
3
1
1
13
1
1
1
1
1
4
1
4
12
1
1
1
1
2
1
2
1
1
5
3
7
1
1
1
1
1
2
1
5
1
1
1
6
1
1
1
1
1
1
6
1
1
3
1
1
3
1
1
2
1
2
1
1
1
1
2
4
6
1
4
13
1
3
1
1
1
6
2
1
1
3
1
1
8
1
2
1
1
1
1
1
8
1
1
1
2
2
5
1
3
1
1
10
1
1
1
1
6
1
1
1
1
3
2
1
10
2
1
1
1
1
5
1
6
1
1
1
1
4
1
3
1
1
1
1
2
9
2
2
1
3
1
3
2
1
1
1
1
1
2
1
4
2
7
1
1
1
3
1

In [18]:
iteration = 8000
inference_validation_data(workspace, holdout_fold, iteration, filename, cuda)

In [31]:
inference_testing_data(workspace, iteration, filename, cuda)

Write result to ./submissions\iteration=8000\submission.csv
